In [ ]:
import codecs

session = spark.builder.getOrCreate();

storage_account_name = "#STORAGE_ACCOUNT_NAME#"
storage_account_access_key = "#STORAGE_ACCOUNT_KEY#"

session.conf.set(
	"fs.azure.account.key.#STORAGE_ACCOUNT_NAME#.blob.core.windows.net",
	"#STORAGE_ACCOUNT_KEY#"
)

sdf = session.read.text(
	"wasbs://wwi-02@#STORAGE_ACCOUNT_NAME#.blob.core.windows.net/ml/product_seasonality_classifier.onnx"
)

def compare(a, b, encoding="utf8"):
    if isinstance(a, bytes):
        a = a.decode(encoding)
    if isinstance(b, bytes):
        b = b.decode(encoding)
    return a == b

for xs in sdf.take(sdf.count()):
    for x in xs:
        for l in x:
            try:
                arr = bytes(l, 'utf-8')

                newArr = codecs.getencoder('hex')(arr)[0]

                if compare(newArr, b'efbfbd'):
                    newArr = b'b4'

                str3 = str3 + newArr.decode("utf-8")
                
            except:
                print('Skipping' + l)

print(str3)

from azure.storage.blob import BlockBlobService

block_blob_service = BlockBlobService(storage_account_name, storage_account_access_key) 
block_blob_service.create_blob_from_text('wwi-02', '/ml/onnx/model.onnx.hex', str3)
